In [44]:
from pathlib import Path
import pandas as pd
from sklearn.linear_model import LinearRegression
import hvplot.pandas
from bokeh.sampledata.unemployment1948 import data as unemployment_df
from bokeh.sampledata.us_marriages_divorces import data as divorces_df
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin
import plotly.express as px

In [45]:
us_suicides_total = pd.read_csv(r"../Resources/Clean/us_suicides_1985-2021.csv")
russia_suicides_total = pd.read_csv(r"../Resources/Clean/russia_suicides_1985-2016.csv")
japan_suicides_total = pd.read_csv(r"../Resources/Clean/japan_suicides_1985-2016.csv")
canada_suicides_total = pd.read_csv(r"../Resources/Clean/canada_suicides_1985-2016.csv")
germany_suicides_total = pd.read_csv(
    r"../Resources/Clean/germany_suicides_1985-2016.csv"
)
untreated_mental_illness = pd.read_csv(
    r"../Resources/Clean/untreated_mental_illness_total_2018-2019.csv"
).assign(Year="2018-2019")
metrics_988 = pd.read_csv(r"../Resources/Clean/988_performance_metrics.csv")
us_suicide_rates = pd.read_csv(r"../Resources/Clean/us_suicide_rates_1985-2016.csv")
russia_suicide_rates = pd.read_csv(
    r"../Resources/Clean/russia_suicide_rates_1985-2016.csv"
)
paraguay_suicide_rates = pd.read_csv(
    r"../Resources\Clean\paraguay_suicide_rates_1985-2016.csv"
)
us_suicide_rates_99_16 = pd.read_csv(
    r"../Resources/Clean/us_suicide_rates_1999-2016.csv"
)
us_suicides_master = pd.read_csv(
    r"../Resources/Clean/us_suicides_1985-2016(master).csv"
)
states_suicides = pd.read_csv(r"../Resources/Originals/state_suicides_2005-2021.csv")

In [46]:
untreated_mental_illness.set_index("Year", inplace=True)

In [47]:
untreated_mental_illness.hvplot.bar(ylim=(0, 100))

:Bars   [Year,Variable]   (value)

In [48]:
paraguay_suicide_rates.hvplot(x="Year", y="Suicides per 100k", rot=45).opts(
    height=450, width=800
) * paraguay_suicide_rates.hvplot.scatter(
    x="Year", y="Suicides per 100k", rot=45, c="k"
).opts(
    height=450, width=800
) * us_suicide_rates.hvplot(
    x="Year", y="Suicides per 100k", rot=45
).opts(
    height=450, width=800
) * us_suicide_rates.hvplot.scatter(
    x="Year", y="Suicides per 100k", rot=45, c="k"
).opts(
    height=450, width=800
)

:Overlay
   .Curve.I    :Curve   [Year]   (Suicides per 100k)
   .Scatter.I  :Scatter   [Year]   (Suicides per 100k)
   .Curve.II   :Curve   [Year]   (Suicides per 100k)
   .Scatter.II :Scatter   [Year]   (Suicides per 100k)

In [49]:
us_suicides_total.hvplot(x="Year", y="Number of Suicides", rot=45).opts(
    height=450, width=800
) * us_suicides_total.hvplot.scatter(
    x="Year", y="Number of Suicides", rot=45, c="k"
).opts(
    height=450, width=800
)

:Overlay
   .Curve.I   :Curve   [Year]   (Number of Suicides)
   .Scatter.I :Scatter   [Year]   (Number of Suicides)

In [50]:
# Separate the features (Year) and target (Number of Suicides)
X = us_suicides_total[["Year"]]
y = us_suicides_total["Number of Suicides"]

# Create and fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# Generate predictions using the fitted model
y_pred = model.predict(X)

# Create scatter plots using hvPlot
scatter_actual = us_suicides_total.hvplot.scatter(
    x="Year", y="Number of Suicides", rot=37
)
scatter_predicted = pd.DataFrame(
    {"Year": us_suicides_total["Year"], "Predicted Suicides": y_pred}
).hvplot.line(x="Year", y="Predicted Suicides", color="red")

# Combine the scatter plots
scatter_actual * scatter_predicted

:Overlay
   .Scatter.I :Scatter   [Year]   (Number of Suicides)
   .Curve.I   :Curve   [Year]   (Predicted Suicides)

In [51]:
metrics_988 = metrics_988.rename(columns={"Unnamed: 0": "Month"})
metrics_988 = metrics_988.iloc[::-1]
metrics_988

,Month,Routed Calls,Routed Chats,Routed Texts,Total Routed,Total Answered Percentage
10,July 2022,245905,75441,33279,354625,84
9,August 2022,256398,64115,40627,361140,88
8,September 2022,239135,66128,45366,350629,88
7,October 2022,230928,68425,49992,349345,91
6,November 2022,222285,72134,52922,347341,92
5,December 2022,241555,70616,59484,371655,91
4,January 2023,248634,71058,64379,384071,91
3,February 2023,226634,59225,60365,346224,92
2,March 2023,243417,61146,64259,371822,92
1,April 2023,242995,61082,64859,368936,92


In [52]:
metrics_988.sum()
# routed_calls_pct = (2660806 / 4008282) * 100
# routed_chats_pct = (737877 / 4008282) * 100
# routed_texts_pct = (606599 / 4008282) * 100

# data988 = {
#     "Interaction Type": ["Routed Calls", "Routed Chats", "Routed Texts"],
#     "Percentage": [routed_calls_pct, routed_chats_pct, routed_texts_pct]
# }

# pct_988 = pd.DataFrame(data988)
# # pct_988.to_csv(r"../Resources/Clean/988_pcts.csv")
# pct_988

Month                        July 2022August 2022September 2022October 2022...
Routed Calls                                                           2660806
Routed Chats                                                            737877
Routed Texts                                                            606599
Total Routed                                                           4008282
Total Answered Percentage                                                  994
dtype: object

In [53]:
metrics_988.hvplot.area(x="Month", width=1200, height=400, yformatter='%.0f')

:NdOverlay   [Variable]
   :Area   [Month]   (value,Baseline)

In [54]:
X = us_suicide_rates_99_16[["Year"]]
y = us_suicide_rates_99_16["Suicides per 100k"]

# Create and fit a linear regression model
model = LinearRegression()
model.fit(X, y)

# Generate predictions using the fitted model
y_pred = model.predict(X)

# Create scatter plots using hvPlot
scatter_actual = us_suicide_rates_99_16.hvplot.scatter(
    x="Year", y="Suicides per 100k", rot=37
)
scatter_predicted = pd.DataFrame(
    {"Year": us_suicide_rates_99_16["Year"], "Predicted Suicides": y_pred}
).hvplot.line(x="Year", y="Predicted Suicides", color="red")

# Combine the scatter plots
scatter_actual * scatter_predicted

:Overlay
   .Scatter.I :Scatter   [Year]   (Suicides per 100k)
   .Curve.I   :Curve   [Year]   (Predicted Suicides)

In [55]:
us_suicides_master.drop(["Unnamed: 0"], axis=1, inplace=True)

In [56]:
us_suicides_master.set_index("Country")

,Year,Sex,Age,Number of Suicides,Suicides per 100k,GDP for Year ($),GDP per Capita ($),Generation
Country,,,,,,,,
United States,1985,male,75+ years,2177,53.57,"4,346,734,000,000",19693,G.I. Generation
United States,1985,male,55-74 years,5302,29.50,"4,346,734,000,000",19693,G.I. Generation
United States,1985,male,25-34 years,5134,24.46,"4,346,734,000,000",19693,Boomers
United States,1985,male,35-54 years,6053,22.77,"4,346,734,000,000",19693,Silent
United States,1985,male,15-24 years,4267,21.38,"4,346,734,000,000",19693,Generation X
...,...,...,...,...,...,...,...,...
United States,2015,female,25-34 years,1444,6.70,"18,120,714,000,000",60387,Millenials
United States,2015,female,15-24 years,1132,5.23,"18,120,714,000,000",60387,Millenials
United States,2015,female,75+ years,540,4.58,"18,120,714,000,000",60387,Silent


In [57]:
unemployment_df = unemployment_df.set_index("Year").drop("Annual", axis=1).transpose()
unemployment_df.hvplot.heatmap(
    x="columns",
    y="index",
    title="US Unemployment 1948—2016",
    cmap=[
        "#75968f",
        "#a5bab7",
        "#c9d9d3",
        "#e2e2e2",
        "#dfccce",
        "#ddb7b1",
        "#cc7878",
        "#933b41",
        "#550b1d",
    ],
    xaxis="top",
    rot=70,
    width=1000,
    height=500,
).opts(fontsize={"title": 10, "xticks": 5, "yticks": 5})

:HeatMap   [columns,index]   (value)

In [58]:
us_suicide_rates.hvplot(x="Year", y="Suicides per 100k")

:Curve   [Year]   (Suicides per 100k)

In [59]:
divorces_df.tail(27).hvplot(
    x="Year", y="Divorces_per_1000", color="maroon"
) + us_suicide_rates.head(27).hvplot(x="Year", y="Suicides per 100k")

:Layout
   .Curve.I  :Curve   [Year]   (Divorces_per_1000)
   .Curve.II :Curve   [Year]   (Suicides per 100k)

In [60]:
us_suicides_master.head()

,Country,Year,Sex,Age,Number of Suicides,Suicides per 100k,GDP for Year ($),GDP per Capita ($),Generation
0,United States,1985,male,75+ years,2177,53.57,"4,346,734,000,000",19693,G.I. Generation
1,United States,1985,male,55-74 years,5302,29.50,"4,346,734,000,000",19693,G.I. Generation
2,United States,1985,male,25-34 years,5134,24.46,"4,346,734,000,000",19693,Boomers
3,United States,1985,male,35-54 years,6053,22.77,"4,346,734,000,000",19693,Silent
4,United States,1985,male,15-24 years,4267,21.38,"4,346,734,000,000",19693,Generation X


In [61]:
us_suicides_males = us_suicides_master[~(us_suicides_master["Sex"] == "female")]
us_suicides_females = us_suicides_master[~(us_suicides_master["Sex"] == "male")]
us_suicides_males = (
    us_suicides_males.groupby("Year")["Suicides per 100k"].sum().reset_index()
)
us_suicides_females = (
    us_suicides_females.groupby("Year")["Suicides per 100k"].sum().reset_index()
)
us_suicides_males.hvplot(x="Year", y="Suicides per 100k") * us_suicides_females.hvplot(
    x="Year", y="Suicides per 100k"
)

:Overlay
   .Curve.I  :Curve   [Year]   (Suicides per 100k)
   .Curve.II :Curve   [Year]   (Suicides per 100k)